# Implementing Retries and Timeouts

- External services can be slow or unreliable, causing scripts to hang or fail unexpectedly.
- Timeouts and retries help ensure your automation scripts remain responsive and resilient.

## Timeouts

- By default, `requests` may wait indefinitely for a response, which is risky in automation.
- Use the `timeout` parameter with a single value for both connect and read, or a tuple `(connect, read)` for fine-grained control.
- A `ConnectTimeout` is raised if the connection can’t be established in time; a `ReadTimeout` is raised if data stops arriving within the read timeout.

In [1]:
GITHUB_ENDPOINT = "https://api.github.com"
HTTPBIN_ENDPOINT = "https://httpbin.org"

In [2]:
import requests
import time 

delay_url = f"{HTTPBIN_ENDPOINT}/delay/5"  # Simulate a delay of 5 seconds

start = time.perf_counter()
try:
    res = requests.get(delay_url, timeout=2)  # Set a timeout of 2 second
    print(f"Completed in {time.perf_counter() - start:.2f} seconds, status code: {res.status_code}")
except (
    requests.exceptions.ConnectTimeout, 
    requests.exceptions.ReadTimeout
    ) as timeout_err:
    print(f"Timeout after: {time.perf_counter() - start:.2f} seconds,{timeout_err}")    

Timeout after: 2.43 seconds,HTTPSConnectionPool(host='httpbin.org', port=443): Read timed out. (read timeout=2)


## Retries

- Transient issues like network blips or server overloads may cause requests to fail temporarily.
- Implement a simple retry loop that catches errors, retries on server-side (5xx) errors or network exceptions, and breaks on success or client errors.
- Use a fixed delay between retries for simplicity, or an exponential backoff for a more robust approach. 
- Avoid retrying non-idempotent operations.

In [6]:
import requests 
import time 

flaky_url = f"{HTTPBIN_ENDPOINT}/status/200,500,503"  # Simulate a server error

max_retries = 3  # Maximum number of retries
delay = 2 

for attempt in range(1, max_retries + 1):
    print(f"Attempt {attempt}/{max_retries}...")

    try:
        res = requests.get(flaky_url, timeout=10)
        res.raise_for_status()
        print(f"Completed in {time.perf_counter() - start:.2f} seconds, status code: {res.status_code}")
        break
    except requests.exceptions.HTTPError as err:
        if err.response.status_code < 500:
            print(f"Client error occurred, status code: {err.response.status_code} Skipping retry...")
            break
        else:
            print(f"Server error occurred, status code: {err.response.status_code}")
    if attempt < max_retries:
        print(f"Retrying in {delay} seconds...")
        time.sleep(delay)    
else:
    print(f"All {max_retries} attempts failed.")

Attempt 1/3...
Server error occurred, status code: 500
Retrying in 2 seconds...
Attempt 2/3...
Completed in 1914.03 seconds, status code: 200


## Exponential Backoff with Jitter

- Fixed delays can overwhelm a recovering server if many clients retry simultaneously.
- Exponential backoff increases the wait time after each failure (e.g., 1s, 2s, 4s...).
- Adding jitter (a small random offset) prevents synchronized retry spikes.

## Common Pitfalls & How to Avoid Them

- Forgetting to set timeouts can cause scripts to hang indefinitely; always use `timeout`.
- Retrying client errors (4xx) usually won’t help; only retry transient server errors (5xx) or network issues.
- Retrying non-idempotent operations (e.g., POST) can cause duplicate actions; limit retries to safe methods.
- Fixed retry delays can lead to synchronized retry spikes; use exponential backoff with jitter for production scenarios.